<a href="https://colab.research.google.com/github/KevinWang676/Bark-Voice-Cloning/blob/main/Voice_Cloning_for_Chinese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 中文声音克隆 Voice Cloning for Chinese

## 配置环境 Set up

In [ ]:
! nvidia-smi # 需要使用GPU运行

In [1]:
! pip install openai-whisper
! pip install modelscope
! pip install tts-autolabel -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
! pip install typeguard==2.3.1
! pip install sox
! pip install bitstring
! pip install kantts -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
! pip install pytorch_wavelets
! pip install tensorboardX
! git clone https://github.com/fbcotter/pytorch_wavelets
! pip install matplotlib
! pip install numpy==1.22.0

# 点击左下角终端，在终端中依次执行三行代码：(1) apt-get install sox (2) cd pytorch_wavelets 和 (3) pip install .


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 19.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796910 sha256=47694d985a7ddcfdff6f19538e54918ce2331170c6e3cf987d51766ab1fe62a4
  Stored in directory: /root/.cache/pip/wheels/b2/13/5f/fe8245f6dc59df505879da4b2129932e342f02a80e6b87f27d
Successfully built openai-whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 kB 29.4 MB/s eta 

Looking in links: https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 177.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 95.7 MB/s et

  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.0.0
    Uninstalling typeguard-4.0.0:
      Successfully uninstalled typeguard-4.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
Looking in links: https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.2/147.2 kB 239.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 31.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pysptk: filename=pysptk-0.2.0-cp310-cp310-linux_x86_64.whl size=1282790 sha256=5701c8443ec3ba751d94d101ecf8a440b8ca58334c9708c0867fa01ab5c8863b
  Stored in directory: /root/.cache/pip/wheels/a7/2a/ed/6a829dcdfd2a67af80144fcdfc9c3f31eac108fafba4fceb59
Successfully built pysp

In [ ]:
import os
os._exit(00) # 重启notebook

In [1]:
import sox

## 选做、实测无影响：在终端中执行
cd ~

apt install zip

wget https://isv-data.oss-cn-hangzhou.aliyuncs.com/ics/MaaS/TTS/download_files/nltk_data.zip

unzip nltk_data.zip


## 对音频切片处理

In [6]:
import subprocess
from pathlib import Path
import librosa
from scipy.io import wavfile
import numpy as np
import torch
import csv
import whisper

def split_long_audio(model, filepaths, character_name, save_dir="data_dir", out_sr=44100):
    if isinstance(filepaths, str):
        filepaths = [filepaths]

    for file_idx, filepath in enumerate(filepaths):

        save_path = Path(save_dir) / character_name
        save_path.mkdir(exist_ok=True, parents=True)

        print(f"Transcribing file {file_idx}: '{filepath}' to segments...")
        result = model.transcribe(filepath, word_timestamps=True, task="transcribe", beam_size=5, best_of=5)
        segments = result['segments']

        wav, sr = librosa.load(filepath, sr=None, offset=0, duration=None, mono=True)
        wav, _ = librosa.effects.trim(wav, top_db=20)
        peak = np.abs(wav).max()
        if peak > 1.0:
            wav = 0.98 * wav / peak
        wav2 = librosa.resample(wav, orig_sr=sr, target_sr=out_sr)
        wav2 /= max(wav2.max(), -wav2.min())

        for i, seg in enumerate(segments):
            start_time = seg['start']
            end_time = seg['end']
            wav_seg = wav2[int(start_time * out_sr):int(end_time * out_sr)]
            wav_seg_name = f"{character_name}_{file_idx}_{i}.wav"
            out_fpath = save_path / wav_seg_name
            wavfile.write(out_fpath, rate=out_sr, data=(wav_seg * np.iinfo(np.int16).max).astype(np.int16))

In [8]:
whisper_size = "medium"
whisper_model = whisper.load_model(whisper_size)

100%|█████████████████████████████████████| 1.42G/1.42G [00:31<00:00, 49.0MiB/s]


In [9]:
split_long_audio(whisper_model, "filename.wav", "test", "dataset_raw") # 请在{filename}处填写您上传的wav文件名

Transcribing file 0: 'nana_speech.wav' to segments...


## 需要新建三个文件夹，分别是："test_wavs", "output_training_data", "pretrain_work_dir"

In [10]:
! mv  -v ./dataset_raw/test/* ./test_wavs/

renamed './dataset_raw/test/test_0_0.wav' -> './test_wavs/test_0_0.wav'
renamed './dataset_raw/test/test_0_10.wav' -> './test_wavs/test_0_10.wav'
renamed './dataset_raw/test/test_0_11.wav' -> './test_wavs/test_0_11.wav'
renamed './dataset_raw/test/test_0_12.wav' -> './test_wavs/test_0_12.wav'
renamed './dataset_raw/test/test_0_13.wav' -> './test_wavs/test_0_13.wav'
renamed './dataset_raw/test/test_0_14.wav' -> './test_wavs/test_0_14.wav'
renamed './dataset_raw/test/test_0_15.wav' -> './test_wavs/test_0_15.wav'
renamed './dataset_raw/test/test_0_16.wav' -> './test_wavs/test_0_16.wav'
renamed './dataset_raw/test/test_0_17.wav' -> './test_wavs/test_0_17.wav'
renamed './dataset_raw/test/test_0_18.wav' -> './test_wavs/test_0_18.wav'
renamed './dataset_raw/test/test_0_19.wav' -> './test_wavs/test_0_19.wav'
renamed './dataset_raw/test/test_0_1.wav' -> './test_wavs/test_0_1.wav'
renamed './dataset_raw/test/test_0_2.wav' -> './test_wavs/test_0_2.wav'
renamed './dataset_raw/test/test_0_3.wav' ->

In [11]:
from modelscope.tools import run_auto_label

2023-07-07 20:56:58,453 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-07-07 20:56:58,456 - modelscope - INFO - TensorFlow version 2.12.0 Found.
2023-07-07 20:56:58,457 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2023-07-07 20:56:58,459 - modelscope - INFO - No valid ast index found from /root/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2023-07-07 20:56:58,509 - modelscope - INFO - Loading done! Current index file version is 1.7.1, with md5 831b00263cf29df16d1aa0e4d51c3ef1 and a total number of 861 components indexed


In [12]:
input_wav = "./test_wavs/"
output_data = "./output_training_data/"

ret, report = run_auto_label(input_wav=input_wav, work_dir=output_data, resource_revision="v1.0.5")

2023-07-07 20:57:00,082 - modelscope - INFO - Use user-specified model revision: v1.0.5
Downloading: 100%|██████████| 185/185 [00:00<00:00, 981kB/s]
Downloading: 100%|██████████| 1.02G/1.02G [00:23<00:00, 47.0MB/s]
Downloading: 100%|██████████| 6.27k/6.27k [00:00<00:00, 2.25MB/s]


2023-07-07 20:57:38
wav_preprocess start...
---  new folder...  ---
---  OK  ---


100%|██████████| 20/20 [00:00<00:00, 100.43it/s]

wav cut by vad start...



100%|██████████| 19/19 [00:02<00:00,  7.45it/s]


...


100%|██████████| 19/19 [00:00<00:00, 136.18it/s]

Convert align info to interval files....
---  There is this folder!  ---
test_0_0.ali
test_0_2.ali
test_0_5.ali
test_0_18.ali
test_0_10.ali
test_0_9.ali
test_0_1.ali
test_0_3.ali
test_0_4.ali
test_0_12.ali
test_0_17.ali
test_0_16.ali
test_0_11.ali
test_0_7.ali
test_0_13.ali
test_0_14.ali
test_0_6.ali
test_0_8.ali
test_0_19.ali
Modify sil to sp in interval....


modify interval er phone.
---  New folder /content/output_training_data/interval...  ---
---  OK  ---
qualification review.
prosody sillence detect.
---  New folder /content/output_training_data/prosody...  ---
---  OK  ---
average silence duration: 0.1726315789473685


100%|██████████| 38/38 [00:00<00:00, 6918.89it/s]

Write prosody file
0 "mismatch" sentences

Auto labeling info: stage 1 | develop mode 0 | gender:female | score 10.000000 | retcode 0
labeling report:
stage 1 | develop mode 0 | gender female | score 10.000000 | retcode 0
qulification report:
credit score: 10.000000
qualified score: 3.000000
normalized snr: 35.000000
abandon utt snr threshold: 10.000000
snr score ration: 0.500000
interval score ration: 0.500000
data qulificaion report:


## 训练 Training

In [13]:
from modelscope.metainfo import Trainers
from modelscope.trainers import build_trainer
from modelscope.utils.audio.audio_utils import TtsTrainType

pretrained_model_id = 'damo/speech_personal_sambert-hifigan_nsf_tts_zh-cn_pretrain_16k'

dataset_id = "./output_training_data/"
pretrain_work_dir = "./pretrain_work_dir/"

# 训练信息，用于指定需要训练哪个或哪些模型，这里展示AM和Vocoder模型皆进行训练
# 目前支持训练：TtsTrainType.TRAIN_TYPE_SAMBERT, TtsTrainType.TRAIN_TYPE_VOC
# 训练SAMBERT会以模型最新step作为基础进行finetune
train_info = {
    TtsTrainType.TRAIN_TYPE_SAMBERT: {  # 配置训练AM（sambert）模型
        'train_steps': 202,               # 训练多少个step
        'save_interval_steps': 200,       # 每训练多少个step保存一次checkpoint
        'log_interval': 10               # 每训练多少个step打印一次训练日志
    }
}

# 配置训练参数，指定数据集，临时工作目录和train_info
kwargs = dict(
    model=pretrained_model_id,                  # 指定要finetune的模型
    model_revision = "v1.0.6",
    work_dir=pretrain_work_dir,                 # 指定临时工作目录
    train_dataset=dataset_id,                   # 指定数据集id
    train_type=train_info                       # 指定要训练类型及参数
)

trainer = build_trainer(Trainers.speech_kantts_trainer,
                        default_args=kwargs)

trainer.train()


2023-07-07 21:01:23,271 - modelscope - INFO - Use user-specified model revision: v1.0.6
Downloading: 100%|██████████| 4.07k/4.07k [00:00<00:00, 1.38MB/s]
2023-07-07 21:01:25,787 - modelscope - INFO - Use user-specified model revision: v1.0.6
Downloading: 100%|██████████| 138k/138k [00:00<00:00, 258kB/s]
Downloading: 100%|██████████| 288k/288k [00:00<00:00, 406kB/s]
Downloading: 100%|██████████| 327k/327k [00:00<00:00, 462kB/s]
Downloading: 100%|██████████| 535k/535k [00:00<00:00, 602kB/s]
Downloading: 100%|██████████| 506/506 [00:00<00:00, 1.99MB/s]
Downloading: 100%|██████████| 1.69M/1.69M [00:01<00:00, 1.41MB/s]
Downloading: 100%|██████████| 452k/452k [00:00<00:00, 512kB/s]
Downloading: 100%|██████████| 998k/998k [00:01<00:00, 943kB/s]
Downloading: 100%|██████████| 866M/866M [00:26<00:00, 34.7MB/s]
Downloading: 100%|██████████| 150M/150M [00:10<00:00, 14.6MB/s]
Downloading: 100%|██████████| 3.82k/3.82k [00:00<00:00, 1.61MB/s]
Downloading: 100%|██████████| 2.49k/2.49k [00:00<00:00, 10

TextScriptConvertor.process:
Save script to: ./pretrain_work_dir/data/Script.xml
TextScriptConvertor.process:
Save metafile to: ./pretrain_work_dir/data/raw_metafile.txt
[AudioProcessor] Initialize AudioProcessor.
[AudioProcessor] config params:
[AudioProcessor] wav_normalize: True
[AudioProcessor] trim_silence: True
[AudioProcessor] trim_silence_threshold_db: 60
[AudioProcessor] preemphasize: False
[AudioProcessor] sampling_rate: 16000
[AudioProcessor] hop_length: 200
[AudioProcessor] win_length: 1000
[AudioProcessor] n_fft: 2048
[AudioProcessor] n_mels: 80
[AudioProcessor] fmin: 0.0
[AudioProcessor] fmax: 8000.0
[AudioProcessor] phone_level_feature: True
[AudioProcessor] se_feature: True
[AudioProcessor] norm_type: mean_std
[AudioProcessor] max_norm: 1.0
[AudioProcessor] symmetric: False
[AudioProcessor] min_level_db: -100.0
[AudioProcessor] ref_level_db: 20
[AudioProcessor] num_workers: 16
[AudioProcessor] Amplitude normalization started
Volume statistic proceeding...



100%|██████████| 19/19 [00:00<00:00, 41.09it/s]


Average amplitude RMS : 0.17481673684210527
Volume statistic done.
Volume normalization proceeding...


100%|██████████| 19/19 [00:00<00:00, 1475.75it/s]

...

100%|██████████| 1/1 [00:01<00:00,  1.01s/it]

Epoch 300 finished



  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 301 finished



2023-07-07 21:07:15,709 - modelscope - INFO - SAMBERT training spent: 0.05 hours

2023-07-07 21:07:15,713 - modelscope - INFO - skip HIFIGAN training...


## 推理 Inference

In [14]:
import os
from modelscope.models.audio.tts import SambertHifigan
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

model_dir = os.path.abspath("./pretrain_work_dir")

custom_infer_abs = {
    'voice_name':
    'F7',
    'am_ckpt':
    os.path.join(model_dir, 'tmp_am', 'ckpt'),
    'am_config':
    os.path.join(model_dir, 'tmp_am', 'config.yaml'),
    'voc_ckpt':
    os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan', 'ckpt'),
    'voc_config':
    os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan',
             'config.yaml'),
    'audio_config':
    os.path.join(model_dir, 'data', 'audio_config.yaml'),
    'se_file':
    os.path.join(model_dir, 'data', 'se', 'se.npy')
}
kwargs = {'custom_ckpt': custom_infer_abs}

model_id = SambertHifigan(os.path.join(model_dir, "orig_model"), **kwargs)

inference = pipeline(task=Tasks.text_to_speech, model=model_id)
output = inference(input="大家好呀，欢迎使用滔滔智能的声音克隆产品！") # 在这里编辑想要输出的文本

import IPython.display as ipd
ipd.Audio(output["output_wav"], rate=16000)


2023-07-07 21:08:41,643 - modelscope - INFO - am_config=/content/pretrain_work_dir/tmp_am/config.yaml voc_config=/content/pretrain_work_dir/orig_model/basemodel_16k/hifigan/config.yaml
2023-07-07 21:08:41,646 - modelscope - INFO - audio_config=/content/pretrain_work_dir/data/audio_config.yaml
2023-07-07 21:08:41,648 - modelscope - INFO - am_ckpts=OrderedDict([(2400000, '/content/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400000.pth'), (2400300, '/content/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400300.pth')])
2023-07-07 21:08:41,649 - modelscope - INFO - voc_ckpts=OrderedDict([(2400000, '/content/pretrain_work_dir/orig_model/basemodel_16k/hifigan/ckpt/checkpoint_2400000.pth')])
2023-07-07 21:08:41,651 - modelscope - INFO - se_path=/content/pretrain_work_dir/data/se/se.npy se_model_path=/content/pretrain_work_dir/orig_model/se.onnx
2023-07-07 21:08:41,653 - modelscope - INFO - mvn_path=/content/pretrain_work_dir/orig_model/mvn.npy
2023-07-07 21:08:45,999 - modelscope - WARNING - No prep

Removing weight norm...


## 打包训练好的模型，可下载到本地

In [15]:
!zip -r /content/model.zip /content/pretrain_work_dir

  adding: content/pretrain_work_dir/ (stored 0%)
  adding: content/pretrain_work_dir/tmp_voc/ (stored 0%)
  adding: content/pretrain_work_dir/tmp_am/ (stored 0%)
  adding: content/pretrain_work_dir/tmp_am/config.yaml (deflated 61%)
  adding: content/pretrain_work_dir/tmp_am/ckpt/ (stored 0%)
  adding: content/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400300.pth (deflated 10%)
  adding: content/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400000.pth (deflated 10%)
  adding: content/pretrain_work_dir/tmp_am/log/ (stored 0%)
  adding: content/pretrain_work_dir/tmp_am/log/events.out.tfevents.1688763841.ac082646fefb (deflated 73%)
  adding: content/pretrain_work_dir/data/ (stored 0%)
  adding: content/pretrain_work_dir/data/frame_uv/ (stored 0%)
  adding: content/pretrain_work_dir/data/frame_uv/test_0_0.npy (deflated 92%)
  adding: content/pretrain_work_dir/data/frame_uv/test_0_10.npy (deflated 84%)
  adding: content/pretrain_work_dir/data/frame_uv/test_0_8.npy (deflated 90%)
  adding: content

In [17]:
from google.colab import files
files.download("/content/model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>